In [7]:
from functions import *
import cv2
import random

list_cam_path = [
    './data/6p-c0.avi',
    './data/6p-c1.avi',
    './data/6p-c2.avi',
    './data/6p-c3.avi'
]

In [ ]:
color_list = random.choices(range(256), k=3*1000)
color_list = np.array(color_list).reshape(-1, 3).tolist()

map_img = cv2.imread('./homography/map.jpg')
height, width = map_img.shape[:2]
print(f"Map size: width={width}, height={height}")
print("Calculate crop...")
min_x_crop, min_y_crop, T, list_H = caculate_crop(width, height)
print("Done calculate crop")
cap0 = cv2.VideoCapture(list_cam_path[0])
cap1 = cv2.VideoCapture(list_cam_path[1])
cap2 = cv2.VideoCapture(list_cam_path[2])
cap3 = cv2.VideoCapture(list_cam_path[3])

fps = int(cap0.get(cv2.CAP_PROP_FPS))

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('./mapping_output.avi', fourcc, fps, (int(width), int(height)))

model_reid = PersonReIDModel(img_feature_dim=256, his_feature_dim=128, n_split=8, dropout=0.33)
model_reid.load_state_dict(torch.load('./reid_module/6p_4c/models/best.pth'))

model_track_c0 = YOLO('./detection_module/detection_weight/best.pt')
model_track_c1 = YOLO('./detection_module/detection_weight/best.pt')
model_track_c2 = YOLO('./detection_module/detection_weight/best.pt')
model_track_c3 = YOLO('./detection_module/detection_weight/best.pt')

person_data = create_data_id_person('./reid_module/id_person/6p_4c')

print("Start mapping...")
frame_idx = 0
while cap0.isOpened() and cap1.isOpened() and cap2.isOpened() and cap3.isOpened():
    ret0, frame0 = cap0.read()
    ret1, frame1 = cap1.read()
    ret2, frame2 = cap2.read()
    ret3, frame3 = cap3.read()
    
    if not (ret0 and ret1 and ret2 and ret3):
        break
    if frame_idx % (fps*0.2) == 0:
        detections = []
        detection1 = Track_Cam(frame0, model_track_c0, model_reid, person_data, cam_id=0)
        detection1 = transform_points(detection1, T, list_H[0])
        detection1 = transform_points_mapping(detection1, min_x_crop, min_y_crop)
        detections.append(detection1)
        
        detection2 = Track_Cam(frame1, model_track_c1, model_reid, person_data, cam_id=1)
        detection2 = transform_points(detection2, T, list_H[1])
        detection2 = transform_points_mapping(detection2, min_x_crop, min_y_crop)
        detections.append(detection2)
        
        detection3 = Track_Cam(frame2, model_track_c2, model_reid, person_data, cam_id=2)
        detection3 = transform_points(detection3, T, list_H[2])
        detection3 = transform_points_mapping(detection3, min_x_crop, min_y_crop)
        detections.append(detection3)
        
        detection4 = Track_Cam(frame3, model_track_c3, model_reid, person_data, cam_id=3)
        detection4 = transform_points(detection4, T, list_H[3])
        detection4 = transform_points_mapping(detection4, min_x_crop, min_y_crop)
        detections.append(detection4)
        
        detection = filter_point(detections, threshold=90)
        print(detection)
        crop = plot_point(color_list, detection, map_img.copy())
        out.write(crop)
    frame_idx += 1
cap0.release()
cap1.release()
cap2.release()
cap3.release()
out.release()
print("Mapping done. Output saved to mapping_output.avi")

Map size: width=1920, height=1080
Calculate crop...
Done calculate crop
Start mapping...
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[{'id': 4, 'center': array([     2042.8,      1242.2], dtype=float32), 'cam_id': 1}]
[{'id': 4, 'center': array([     2042.9,      1239.6], dtype=float32), 'cam_id': 1}]
[{'id': 4, 'center': array([     2039.3,      1238.1], dtype=float32), 'cam_id': 1}]
[{'id': 4, 'center': array([     2033.7,        1239], dtype=float32), 'cam_id': 1}]
[{'id': 4, 'center': array([       2030,      1238.9], dtype=float32), 'cam_id': 1}]
[{'id': 4, 'center': array([     2022.7,        1242], dtype=float32), 'cam_id': 1}]
[{'id': 4, 'center': array([     2013.8,      1246.3], dtype=float32), 'cam_id': 1}]
[{'id': 4, 'center': array([     2010.2,      1246.9], dtype=float32), 'cam_id': 1}]
[{'id': 4, 'center': array([     2008.2,      1245.3], dtype=float32), 'cam_id': 1}]
[{'id': 4, 'center': array([     2002.6,      1244.5], dtype=float32), 'cam_id': 1}]
[{'id': 4,

KeyboardInterrupt: 